In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
import time
import re
import urllib.request
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
from fake_useragent import UserAgent
from selectorlib import Extractor
import requests
import json
import jsonlines

/home/rehna/dsa_c2/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#! pip install fake-useragent

In [3]:
#! pip install selectorlib

In [4]:
#! pip install jsonlines

In [5]:
def search_amazon(item):

    useragent = UserAgent()
    options = Options()
    options.headless = True
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", useragent.random)
    #browser = webdriver.Firefox(options=options, firefox_profile=profile)
    browser = webdriver.Firefox(firefox_profile=profile)
    
    base_url = 'https://www.amazon.in/'
    search_key = 'macbook'
    
    browser.get(base_url)
    #time.sleep(5)
    searchname = browser.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
    search_key = "macbook"
    searchname.send_keys(search_key)

    search = browser.find_elements_by_xpath('//*[@id="nav-search-submit-button"]')[0]
    search.click()
    head = True
    
    browser.implicitly_wait(5)

    try:
        num_page = browser.find_element_by_xpath('//*[@class="a-pagination"]/li[6]')
    except NoSuchElementException:
        num_page = browser.find_element_by_class_name('a-last').click()

    browser.implicitly_wait(3)

    url_list = []
    images_url=[]
    for i in range(int(num_page.text)):
        page_ = i + 1
        url_list.append(browser.current_url)
        
        #extracting images
        
        #images = browser.find_elements_by_tag_name('img')
        #count=0
        #for image in images:
        #    count+=1
        #    print(image.get_attribute('src'))
         #   images_url.append(image.get_attribute('src'))
            # download the image
            #src=image.get_attribute('src')
            #urlretrieve(src, 'image'+'_'+str(count)+".png")
        
        browser.implicitly_wait(4)
        click_next = browser.find_element_by_class_name('a-last').click()
        print("Page " + str(page_) + " grabbed")

    browser.quit()
    
    #with open('image_url.txt','w') as f:
    #    for i in images_url:
    #        f.write('%s\n' % i)

    with open('search_results_urls.txt', 'w') as filehandle:
        for result_page in url_list:
            filehandle.write('%s\n' % result_page)

    print("---DONE---")

In [6]:
def scrape(url):
    

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.in/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url)
    print(r.status_code)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create
    return e.extract(r.text)

yml_file='search_results.yml'
import sys

if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

e = Extractor.from_yaml_file(yml_file)

search_amazon('Macbook') # <------ search query goes here.

Page 1 grabbed
Page 2 grabbed
Page 3 grabbed
Page 4 grabbed
Page 5 grabbed
Page 6 grabbed
Page 7 grabbed
Page 8 grabbed
Page 9 grabbed
Page 10 grabbed
Page 11 grabbed
---DONE---


In [7]:
def extract_pages(file):

    # product_data = []
    with open(file,'r') as urllist, open('search_results_output.jsonl','w') as outfile:
        for url in urllist.read().splitlines():
            data = scrape(url)
            if data:
                for product in data['products']:
                    product['search_url'] = url
                    print("Saving Product: %s"%product['title'].encode('utf8'))
                    json.dump(product,outfile)
                    outfile.write("\n")
                    # sleep(5)
file='search_results_urls.txt'
extract_pages(file)  

503
Page https://www.amazon.in/s?k=macbook&ref=nb_sb_noss was blocked by Amazon. Please try using better proxies

503
Page https://www.amazon.in/s/?k=macbook+air+m1+chip&ref=sugsr_3_6&pd_rd_w=nG8D3&pf_rd_p=206c771a-9255-43c3-91ee-be3d883c7b95&pf_rd_r=B4BJ1GK037ZYGTV2XM85&pd_rd_r=571bbb7a-5501-433f-821c-8eabbb2f0625&pd_rd_wg=AVOud&qid=1623513798 was blocked by Amazon. Please try using better proxies

503
Page https://www.amazon.in/s/?k=ipad&ref=sugsr_3_1&pd_rd_w=cJN2L&pf_rd_p=206c771a-9255-43c3-91ee-be3d883c7b95&pf_rd_r=MXPHMKHG4GSK3QWVZX6N&pd_rd_r=a5e9b58c-4f44-4452-b93f-40e02b0c0e04&pd_rd_wg=DgFCa&qid=1623513801 was blocked by Amazon. Please try using better proxies

503
Page https://www.amazon.in/s/?k=ipad+mini&ref=sugsr_3_10&pd_rd_w=HRn3I&pf_rd_p=206c771a-9255-43c3-91ee-be3d883c7b95&pf_rd_r=7N8A7VFPZ4VKTTN66MQQ&pd_rd_r=afdf0d9d-abcf-4ae0-8844-012fe570e2d4&pd_rd_wg=2D5hM&qid=1623513803 was blocked by Amazon. Please try using better proxies

503
Page https://www.amazon.in/s/?k=ipad+mi

503
Page https://www.amazon.in/s/?k=ipad+mini+case&ref=sugsr_3_3&pd_rd_w=rwRQ5&pf_rd_p=206c771a-9255-43c3-91ee-be3d883c7b95&pf_rd_r=0AE56WKS5WC2397G72FV&pd_rd_r=f68ce8b6-1e58-4c25-8ef1-998df354964d&pd_rd_wg=EeKis&qid=1623513812 was blocked by Amazon. Please try using better proxies

503
Page https://www.amazon.in/s/?k=ipad+mini+cover+case&ref=sugsr_3_2&pd_rd_w=cXyEg&pf_rd_p=206c771a-9255-43c3-91ee-be3d883c7b95&pf_rd_r=3FRPRAXT1EHHERTDWEHD&pd_rd_r=1c01068e-f91a-42bc-a18d-b9d9eda549f8&pd_rd_wg=LZYHT&qid=1623513813 was blocked by Amazon. Please try using better proxies



In [8]:
def create_df():
    df=pd.DataFrame()
    title=[]
    url=[]
    rating=[]
    reviews=[]
    price=[]
    search_url=[]
    with jsonlines.open('search_results_output.jsonl') as f:
        for line in f.iter():
            title.append(line['title'])
            url.append(line['url'])
            price.append(line['price'])
            rating.append(line['rating'])
            reviews.append(line['reviews'])
            search_url.append(line['search_url'])
    df['title']=title
    df['url']=url
    df['rating']=rating
    df['reviews']=reviews
    df['price']=price
    df['search_url']=search_url
    return df

In [15]:
df=create_df()

In [16]:
df.to_csv("scrape.csv")

In [13]:
def scrape2(url):  

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e1.extract(r.text)

# Create an Extractor by reading from the YAML file
file2='selectors.yml'
e1 = Extractor.from_yaml_file(file2)

# product_data = []
with open("urls.txt",'r') as urllist, open('output.jsonl','w') as outfile:
    for url in urllist.read().splitlines():
        data = scrape2(url) 
        if data:
            json.dump(data,outfile)
            outfile.write("\n")
            # sleep(5)

In [14]:
def create_df2_eachproduct():
    df=pd.DataFrame()
    name=[]
    price=[]
    short_description=[]
    images=[]
    number_of_reviews=[]
    variants=[]
    product_description=[]
    sales_rank=[]
    link_to_all_reviews=[]
    with jsonlines.open('output.jsonl') as f:
        for line in f.iter():
            name.append(line['name'])
            price.append(line['price'])
            short_description.append(line['short_description'])
            images.append(line['images'])
            number_of_reviews.append(line['number_of_reviews'])
            variants.append(line['variants'])
            product_description.append(line['product_description'])
            sales_rank.append(line['sales_rank'])
            link_to_all_reviews.append(line['link_to_all_reviews'])
    df['name']=name
    df['price']=price
    df['short_description']=short_description
    df['images']=images
    df['number_of_reviews']=number_of_reviews
    df['variants']=variants
    df['product_description']=product_description
    df['sales_rank']=sales_rank
    df['link_to_all_reviews']=link_to_all_reviews
    return df
create_df2_eachproduct()

,name,price,short_description,images,number_of_reviews,variants,product_description,sales_rank,link_to_all_reviews
0,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None
